# Introdução

## Importação das bibliotecas necessarias

In [1]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys

## Funções auxiliares

In [2]:
def return_sql_query(query_string):
    ret = {
        "token": {
            "token": token
        },
        "sql": {
            "sql": {
                "query":query_string
            }
        }
    }
    return ret

def json_to_df(resp):
    columns = [col['name'] for col in resp.json()['columns']]
    values = resp.json()['rows']  
    return pd.DataFrame(values, columns=columns)

def rows_and_columns_to_df(columns, rows):
    columns_array = [col['name'] for col in columns]
    return pd.DataFrame(rows, columns=columns_array)

def fetch_all_data_from_response(res, initial_records_rows):
    while 'cursor' in res.json():
        res_next = {
            "token": {
                "token": token
            },
            "sql": {
                "sql": {
                    "cursor": res.json()['cursor']
                }
            }
        }
        res = requests.post(os.path.join(url_base,'sql_query'), json=res_next)
        initial_records_rows+=res.json()['rows']
        sys.stdout.write('.')
        sys.stdout.flush()
    print('(y)')
    return initial_records_rows

## Dados de acesso a api

In [3]:
from IPython.display import clear_output
import getpass

token = getpass.getpass('Digite o seu token para acesso a API:')

clear_output()

In [4]:
url_base = "https://bigdata-api.fiocruz.br"

## Bases de dados disponiveis na API atualizadas

In [5]:
params = {
          "token": token
        }
resp = requests.post(url_base+ '/show_tables', json=params)
print(resp.json())

{'databases': ['datasus-sim', 'datasus-sinasc', 'datasus-sih']}


## Busca dos campos existentes na base SIM

In [6]:
%time
sql = {
          "token": {
            "token": token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sim"'}
          }
        }
resp = requests.post(os.path.join(url_base,'sql_query'), json=sql)
#resp.content
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.58 µs
['ACIDTRAB', 'ALTCAUSA', 'ASSISTMED', 'ATESTADO', 'ATESTANTE', 'CAUSABAS', 'CAUSABAS_O', 'CAUSAMAT', 'CB_PRE', 'CIRCOBITO', 'CIRURGIA', 'CODBAIOCOR', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODIFICADO', 'CODINST', 'CODMUNCART', 'CODMUNNATU', 'CODMUNOCOR', 'CODMUNRES', 'COMUNSVOIM', 'CONTADOR', 'CRM', 'DIFDATA', 'DTATESTADO', 'DTCADASTRO', 'DTCADINF', 'DTCADINV', 'DTCONCASO', 'DTCONINV', 'DTINVESTIG', 'DTNASC', 'DTOBITO', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'ESC', 'ESC2010', 'ESCFALAGR1', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTABDESCR', 'ESTCIV', 'EXAME', 'EXPDIFDATA', 'FONTE', 'FONTEINV', 'FONTES', 'FONTESINF', 'GESTACAO', 'GRAVIDEZ', 'HORAOBITO', 'IDADE', 'IDADEMAE', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'LOCOCOR', 'MORTEPARTO', 'NATURAL', 'NECROPSIA', 'NUDIASINF', 'NUDIASOBCO', 'NUDIASOBIN', 'NUMERODN', 'NUMERODO', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'NUMSUS', 'OBITOGRAV', 'OBITOPARTO', 

## Busca dos campos existentes na base SINASC

In [7]:
%time
sql = {
          "token": {
            "token": token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sinasc"'}
          }
        }
resp = requests.post(os.path.join(url_base,'sql_query'), json=sql)
#resp.content
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.78 µs
['APGAR1', 'APGAR5', 'CODANOMAL', 'CODBAINASC', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODINST', 'CODMUNCART', 'CODMUNNASC', 'CODMUNNATU', 'CODMUNRES', 'CODOCUPMAE', 'CODPAISRES', 'CODUFNATU', 'CONSPRENAT', 'CONSULTAS', 'CONTADOR', 'DIFDATA', 'DTCADASTRO', 'DTDECLARAC', 'DTNASC', 'DTNASCMAE', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'DTULTMENST', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTCIVMAE', 'GESTACAO', 'GRAVIDEZ', 'HORANASC', 'IDADEMAE', 'IDADEPAI', 'IDANOMAL', 'KOTELCHUCK', 'LOCNASC', 'MESPRENAT', 'NATURALMAE', 'NOVO', 'NUMERODN', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'ORIGEM', 'PARIDADE', 'PARTO', 'PESO', 'PREFIXODN', 'QTDFILMORT', 'QTDFILVIVO', 'QTDGESTANT', 'QTDPARTCES', 'QTDPARTNOR', 'RACACOR', 'RACACORMAE', 'RACACORN', 'RACACOR_RN', 'SEMAGESTAC', 'SERIESCMAE', 'SEXO', 'STCESPARTO', 'STDNEPIDEM', 'STDNNOVA', 'STTRABPART', 'TPAPRESENT', 'TPDOCRESP', 'TPFUNCRESP', 'TPMETESTIM', 'TPNASCASSI', 'TPROBSON'

# Proporção de óbitos de mulher em idade fértil investigados (SIM)

A ideia inicial era explorar os dados de todo o Brasil do período de 2012 a 2019.
Como a tabela SIH-RD tratada e filtrada que temos é do período de 2018 até 2019 e apenas do Estado de São Paulo, iremos utilizar o mesmo subconjunto.

In [8]:
%time
sql_total_obitos = return_sql_query(
    "SELECT CODMUNRES, ano_obito as Ano, count(*) as Qtd_Obitos_Declarados \
    FROM \"datasus-sim\" WHERE \
    ano_obito>=2012 AND \
    ano_obito<=2019 AND \
    idade_obito_anos>=10 AND \
    idade_obito_anos<=49 AND \
    (OBITOGRAV=1 OR OBITOPUERP=1 ) \
    GROUP BY CODMUNRES, ano_obito")

resp_sql_total_obitos = requests.post(os.path.join(url_base,'sql_query'), json=sql_total_obitos)
record_columns=resp_sql_total_obitos.json()['columns']
record_rows=resp_sql_total_obitos.json()['rows']

fetch_all_data_from_response(resp_sql_total_obitos, record_rows)
print("Record rows length : "+str(len(record_rows)))

df_total_obitos = rows_and_columns_to_df(record_columns, record_rows)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.78 µs
........(y)
Record rows length : 7916


In [9]:
%time   
sql_obitos_di = return_sql_query(
    "SELECT CODMUNRES, ano_obito as Ano, count(*) as Qtd_Obitos_Declarados_Investigados \
    FROM \"datasus-sim\" WHERE \
    ano_obito>=2012 AND \
    ano_obito<=2019 AND \
    idade_obito_anos>=10 AND \
    idade_obito_anos<=49 AND \
    TPPOS LIKE \'%S%\' AND \
    (OBITOGRAV=1 OR OBITOPUERP=1 ) \
    GROUP BY CODMUNRES, ano_obito")
    
resp_sql_obitos_di = requests.post(os.path.join(url_base,'sql_query'), json=sql_obitos_di)
record_columns=resp_sql_obitos_di.json()['columns']
record_rows=resp_sql_obitos_di.json()['rows']

fetch_all_data_from_response(resp_sql_obitos_di, record_rows)
print("Record rows length : "+str(len(record_rows)))

df_obitos_di = rows_and_columns_to_df(record_columns, record_rows)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.3 µs
......(y)
Record rows length : 5328


In [10]:
df_obitos_di.head()

,CODMUNRES,Ano,Qtd_Obitos_Declarados_Investigados
0,110002,2013,1
1,110002,2015,1
2,110002,2016,3
3,110002,2017,1
4,110002,2019,1


In [11]:
df_total_obitos.head()

,CODMUNRES,Ano,Qtd_Obitos_Declarados
0,110001,2015,1
1,110002,2012,2
2,110002,2013,1
3,110002,2014,2
4,110002,2015,3


In [12]:
df_obitos_di['Ano'] = df_obitos_di['Ano'].astype(int)
df_total_obitos['Ano'] = df_total_obitos['Ano'].astype(int)
df_merge = pd.merge(df_obitos_di, df_total_obitos, how='right', on=['CODMUNRES','Ano'])
df_merge = df_merge.fillna(0)
df_merge['Ratio'] = df_merge['Qtd_Obitos_Declarados_Investigados']/(df_merge['Qtd_Obitos_Declarados'])
df_merge.head(10)

,CODMUNRES,Ano,Qtd_Obitos_Declarados_Investigados,Qtd_Obitos_Declarados,Ratio
0,110001,2015,0.0,1,0.000000
1,110002,2012,0.0,2,0.000000
2,110002,2013,1.0,1,1.000000
3,110002,2014,0.0,2,0.000000
4,110002,2015,1.0,3,0.333333
5,110002,2016,3.0,4,0.750000
6,110002,2017,1.0,2,0.500000
7,110002,2019,1.0,1,1.000000
8,110004,2014,0.0,1,0.000000
9,110004,2016,1.0,1,1.000000


In [13]:
%time
df_merge.to_csv('./csv_files/W-25-proporcao_obitos_maternos_declarados_investigados.csv',index=False)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.58 µs
